In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Code taken from wonderful resource by Quantpy, only slightly renamed to only take 
# the necessary code snippets for our purposes
# https://quantpy.com.au/stochastic-volatility-models/heston-model-calibration-to-option-prices/

In [2]:
def charFunc(z, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    a = kappa*theta
    b = kappa+lambd
    rspi = z*rho*sigma*1j
    d = np.sqrt( (z*rho*sigma*1j - b)**2 + (z*1j+z**2)*sigma**2 )
    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)
    # calculate characteristic function by components
    exp1 = np.exp(r*z*1j*tau)
    term2 = S0**(z*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
    return exp1*term2*exp2

In [3]:
def hestonPrice(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    
    P, umax, N = 0, 100, 10000
    dz=umax/N # partition size
    for i in range(1,N):
        z = dz * (2*i + 1)/2
        numerator = np.exp(r*tau)*charFunc(z-1j,*args) - K * charFunc(z,*args)
        denominator = 1j*z*K**(1j*z)
        P += dz * numerator/denominator
    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

In [4]:
targets = {
    1: {95: 6.5757, 100: 2.8223, 105: 0.6335},
    2: {95: 8.1165, 100: 4.3850, 105: 1.7263},
    3: {100: 6.0865, 105: 3.1820, 110: 1.2347},
    4: {100: 7.7710, 105: 4.7369, 110: 2.4165}
}

In [9]:
S0 = 100
r = 0.0411
K = np.asarray([s for q in targets for s in targets[q]])
tau = np.asarray([q*0.25 for q in targets for s in targets[q] ])
P = np.asarray([targets[q][s] for q in targets for s in targets[q]])
params = {"v0": {"x0": 0.01211, "lbub": [0.0121,0.01212]}, 
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.3, "lbub": [1e-2,1]},
          "rho": {"x0": -0.9, "lbub": [-1,-0.7]},
          "lambd": {"x0": 0.03, "lbub": [0,1]},
          }
x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

def sse(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    err = np.sum( (P-hestonPrice(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 /len(P) )
    return err

result = minimize(sse, x0, tol = 1e-3, method='SLSQP', options={'maxiter': 1e4 }, bounds=bnds)
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

/home/gany/miniconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
<ipython-input-2-79388f6db6cc>:11: RuntimeWarning: overflow encountered in exp
  exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)
<ipython-input-2-79388f6db6cc>:12: RuntimeWarning: invalid value encountered in multiply
  return exp1*term2*exp2


(0.0121,
 0.6581171606243393,
 0.034490489997797344,
 0.010000000000063259,
 -0.711475586337558,
 0.6915084263343003)

In [10]:
heston_prices = hestonPrice(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

In [11]:
heston_prices

array([6.78178895, 2.90832777, 0.61800107, 8.53892305, 4.67042252,
       1.88486886, 6.29411431, 3.22631565, 1.10767935, 7.86080954,
       4.59584418, 2.13630139])

In [12]:
print("h_0 = ", v0)
print("rho = ", rho)
print("kappa = ", kappa + lambd)
print("theta = ", kappa * theta / (kappa + lambd))
print("xi = ", sigma)

h_0 =  0.0121
rho =  -0.711475586337558
kappa =  1.3496255869586395
theta =  0.01681857810434961
xi =  0.010000000000063259
